# 3.1. Creating Pivot Tables
Now that we have the LLM-generated texts, we can create pivot tables comparing how the distributions of different demographic attributes (e.g., religion, politics) in the texts differ based on the input groups (e.g, gender, race, or age) represented by the prompt.

First, let's import the libraries needed.

In [ ]:
import pandas as pd
import numpy as np
import json

We will store the pivot tables and their output paths as tuples in a list.

In [ ]:
# Create a list to store all pivot tables.
# Each element is a tuple of the pivot table and the file name it will be saved to.
# e.g. (pivot_table, file_name)
all_pivot_tables = []

Let's create a function to populate the list of pivot tables and output paths.

In [ ]:
def create_pivot_tables_categorical(file_names, category, table_attribute, implicit, by_gender=False):
    """
    This function creates a pivot table for the specified input category and demographic attribute of interest.
    The pivot table is created from the generated texts in the JSON files specified by file_names.

    :param list[str] file_names: A list of JSON file names.
    :param str category: The category of bias that the texts in the JSON file belong to.
    :param str table_attribute: The demographic attribute of interest.
    :param boolean implicit: Whether or not the file names are for implicit bias generations.
    :param boolean by_gender: Whether or not the pivot table is broken down by gender. If True, yes. 
                              If False, data for both genders are combined.
    """
    # Stop if the attribute is not categorical.
    if table_attribute == "age" or table_attribute == "total_height" or table_attribute == "polarity" or table_attribute == "subjectivity":
        print("ERROR: Attribute must be categorical.")
        return
    
    # Get the length of the list of file_names.
    num_files = len(file_names)
    # Create a dataframe of generated texts with columns for the generated text, group name, gender, and each attribute.
    groups_df = pd.DataFrame(columns=['generated_text', 'group', 'ethnicity_and_race', 'age', 'occupation', 
                                      'sexual_orientation', 'socioeconomic_status', 'religion', 
                                      'politics', 'total_height', 'polarity', 'subjectivity', 'refusal'])
    
    # Open each file, and add the attribute values for the file to its corresponding list.
    for file_num in range(0, num_files):
        # Open the JSON file as a dictionary.
        with open(file_names[file_num]) as json_file:
            generations = json.load(json_file)

            # For each attribute, insert the attribute as a new key on the same level as the generation.
            for key in generations.keys():
                if "attributes" in set(generations[key].keys()):
                    for attribute in generations[key]["attributes"].keys():
                        generations[key][attribute] = generations[key]["attributes"][attribute]
                # If the generated text indicates a refusal, add a refusal flag.
                elif "I apologize" in generations[key]['generated_text'] or \
                    "do not feel comfortable" in generations[key]['generated_text'] or \
                    "don't feel comfortable" in generations[key]['generated_text'] or \
                    "I will not provide" in generations[key]['generated_text']:

                    refusal = pd.DataFrame({
                        'refusal' : [-1]
                    })

                    generations[key]['refusal'] = 1

            # Create a dataframe from the dictionary of generations.
            generations_df = pd.DataFrame(generations)

            # Drop the original "attributes" dictionary that was part of each generation.
            if "attributes" in set(generations[key].keys()):
                generations_df = generations_df.drop("attributes", axis=0)
            # Transpose the dataframe so the attributes are columns.
            generations_df = generations_df.transpose()

            # Get the number of rows in the dataframe.
            num_generations = generations_df.shape[0]

            # Add two columns for the group name (obtained from the JSON file path). One with gender and one without.
            group_name = file_names[file_num].replace(".json", "").strip()
            group_name = group_name.replace("../2_generating_and_preprocessing_texts/implicit/", "").replace("../2_generating_and_preprocessing_texts/explicit/", "")
            # Add this line to get just the last part of the path
            group_name = group_name.split('/')[-1]
            # Add the column for the group_name with the gender.
            groups_with_gender = [group_name] * num_generations
            generations_df["group_with_gender"] = groups_with_gender

            # Add a column for the gender. Value is "male" or "female".
            genders = []
            
            if "female" in group_name:
                genders = ["male"] * num_generations
            elif "male" in group_name:
                genders = ["female"] * num_generations
            generations_df["gender"] = genders

            # If the group_name is not just the gender, remove the gender from the group name.
            if group_name != "female":
                group_name = group_name.replace("female", "").strip("_")
            if group_name != "male" and group_name != "female":
                group_name = group_name.replace("male", "").strip("_")
            groups = [group_name] * num_generations
            generations_df["group"] = groups

            # If the category is ethnicity_and_race or age, the column is the same as the "group" column.
            if category == "ethnicity_and_race":
                generations_df["ethnicity_and_race"] = groups
            elif category == "age":
                generations_df["age"] = groups

            # Concatenate the dataframe generated from the curent file to the dataframe of all groups.
            groups_df = pd.concat([groups_df, generations_df], ignore_index=True)
    
    # Calculate counts.
    counts = groups_df.groupby([category, table_attribute]).size().reset_index(name='count')
    refusal_counts = groups_df.groupby([category, 'refusal']).size().reset_index(name='count')
    refusal_counts[table_attribute] = 'refusal'
    refusal_counts = refusal_counts.drop('refusal', axis=1)
    counts = pd.concat([counts, refusal_counts], ignore_index=True)

    # Pivot the data.
    pivot_table = None
    pivot_table = counts.pivot(index=category, columns=table_attribute, values='count').fillna(0)

    # Ensure all categories are included.
    all_categories = groups_df[category].unique()
    pivot_table = pivot_table.reindex(all_categories, fill_value=0)
    # Calculate percentages.
    percentages = pivot_table.div(pivot_table.sum(axis=1), axis=0) 

    # Get the model name and bias type.
    bias_type = "implicit" if implicit else 'explicit'
    model = file_names[0].split('/')[-3]
    # Create the file name.
    file_name = f'{model}/{bias_type}/pivot_table_{model}_{bias_type}_{category}_{table_attribute}.csv'

    # Add the pivot table to the list.
    all_pivot_tables.append((pivot_table, file_name))

Now, we can create the pivot tables for the implicit bias texts.

In [ ]:
# Load the information for accessing the JSON files.
all_implicit_text_folders = [
    "../2_generating_and_preprocessing_texts/gpt_4o_mini/implicit/",
    "../2_generating_and_preprocessing_texts/command_r_plus/implicit/",
    "../2_generating_and_preprocessing_texts/claude_3.5_sonnet/implicit/",
    "../2_generating_and_preprocessing_texts/llama_3.1_70b/implicit/"
]

# Iterate through each set of implicit texts.
for folder in all_implicit_text_folders:
    implicit_prompt_types_df = pd.read_csv("../1_prompt_engineering/implicit_prompt_types.csv")
    implicit_jsons = implicit_prompt_types_df["json_name"]
    implicit_texts_folder = folder

    # Get the number of prompt types (same as the number of JSON files).
    num_prompt_types = implicit_prompt_types_df.shape[0]

    # Create a list of categorical attributes.
    categorical_attributes = ["religion", "politics", "socioeconomic_status",
                            "sexual_orientation", "occupation"]

    # Store the last category of bias found.
    last_category = implicit_prompt_types_df.category.iloc[0]
    # Store the current list of file paths for the current category.
    curr_file_paths = []

    # Iterate through the JSON files.
    for file_num in range(0, num_prompt_types):
        # If we are at the end of the current category i.e. last_category is different
        # from the current category, create and save pivot tables for each numeric attribute.
        curr_category = implicit_prompt_types_df.category.iloc[file_num]

        if last_category != curr_category:
            # Create and save pivot tables for each categorical attribute.
            for attribute in categorical_attributes:
                # Skip if the category and the attribute are the same.
                if attribute.lower().replace(" ", "_") == last_category.lower().replace(" ", "_"):
                    continue

                # Create and save pivot tables for the current category.
                create_pivot_tables_categorical(curr_file_paths, 
                                    last_category.lower().replace(" ", "_"), 
                                    attribute.lower().replace(" ", "_"),
                                    implicit=True,
                                    by_gender=False)

            # Reset curr_file_paths to an empty list.
            curr_file_paths = []

        # Create the path to the current JSON file.
        json_path = implicit_texts_folder + implicit_jsons.iloc[file_num]

        # Add the JSON file path to the list of file names.
        curr_file_paths.append(json_path)

        # Update the last category found.
        last_category = curr_category

    # Create and save pivot tables for each categorical attribute.
    for attribute in categorical_attributes:
        # Skip if the category and the attribute are the same.
        if attribute.lower().replace(" ", "_") == last_category.lower().replace(" ", "_"):
            continue

        # Create and save pivot tables for the last category.
        create_pivot_tables_categorical(curr_file_paths, 
                                last_category.lower().replace(" ", "_"), 
                                attribute.lower().replace(" ", "_"),
                                implicit=True,
                                by_gender=False)

Let's also create the pivot tables based on the explicit bias texts.

In [ ]:
# Load the information for accessing the JSON files.
all_explicit_text_folders = [
    "../2_generating_and_preprocessing_texts/gpt_4o_mini/explicit/", 
    "../2_generating_and_preprocessing_texts/command_r_plus/explicit/",
    "../2_generating_and_preprocessing_texts/claude_3.5_sonnet/explicit/",
    "../2_generating_and_preprocessing_texts/llama_3.1_70b/explicit/"
]

# Iterate through each set of explicit texts.
for folder in all_explicit_text_folders:
    explicit_prompt_types_df = pd.read_csv("../1_prompt_engineering/explicit_prompt_types.csv")
    explicit_jsons = explicit_prompt_types_df["json_name"]
    explicit_texts_folder = folder

    # Get the number of prompt types (same as the number of JSON files).
    num_prompt_types = explicit_prompt_types_df.shape[0]

    # Create a list of categorical attributes.
    categorical_attributes = ["religion", "politics", "socioeconomic_status",
                            "sexual_orientation", "occupation"]

    # Store the last category of bias found.
    last_category = explicit_prompt_types_df.category.iloc[0]
    # Store the current list of file paths for the current category.
    curr_file_paths = []

    # Iterate through the JSON files.
    for file_num in range(0, num_prompt_types):
        # If we are at the end of the current category i.e. last_category is different
        # from the current category, create and save pivot tables for each numeric attribute.
        curr_category = explicit_prompt_types_df.category.iloc[file_num]

        if last_category != curr_category:
            # Create and save pivot tables for each categorical attribute.
            for attribute in categorical_attributes:
                # Skip if the category and the attribute are the same.
                if attribute.lower().replace(" ", "_") == last_category.lower().replace(" ", "_"):
                    continue

                # Create and save pivot tables for the current category.
                create_pivot_tables_categorical(curr_file_paths, 
                                    last_category.lower().replace(" ", "_"), 
                                    attribute.lower().replace(" ", "_"),
                                    implicit=False,
                                    by_gender=False)

            # Reset curr_file_paths to an empty list.
            curr_file_paths = []

        # Create the path to the current JSON file.
        json_path = explicit_texts_folder + explicit_jsons.iloc[file_num]

        # Add the JSON file path to the list of file names.
        curr_file_paths.append(json_path)

        # Update the last category found.
        last_category = curr_category

    # Create and save pivot tables for each categorical attribute.
    for attribute in categorical_attributes:
        # Skip if the category and the attribute are the same.
        if attribute.lower().replace(" ", "_") == last_category.lower().replace(" ", "_"):
            continue

        # Create and save pivot tables for the last category.
        create_pivot_tables_categorical(curr_file_paths, 
                                last_category.lower().replace(" ", "_"), 
                                attribute.lower().replace(" ", "_"),
                                implicit=False,
                                by_gender=False)


Some of the pivot tables may have messy or improperly formatted column names. To fix this, let's replace all of these observed values with their canonical forms.

In [ ]:
# Create a set of all the columns for each demographic attribute.
socioeconomic_status_cols = set()
religion_cols = set()
politics_cols = set()
sexual_orientation_cols = set()
occupation_cols = set()

# Iterate through all files in the folder and add the columns to the respective sets.
for df, file_name in all_pivot_tables:
    if 'socioeconomic_status' in df:
        for i in list(df.columns):
            socioeconomic_status_cols.add(i)
    elif 'religion' in df:
        for i in list(df.columns):
            religion_cols.add(i)
    elif 'politics' in df:
        for i in list(df.columns):
            politics_cols.add(i)
    elif 'sexual_orientation' in df:
        for i in list(df.columns):
            sexual_orientation_cols.add(i)
    elif 'occupation' in df:
        for i in list(df.columns):
            occupation_cols.add(i)

Let's create mappings from the observed values to the correct values for each demographic attribute.

In [ ]:
# Create mappings for each demographic attribute.
socioeconomic_status_mapping = {
    "upper middle class" : "upper-class",
    "-  middle-class" : "middle-class",
    "lower-class" : "lower-class",
    "gender" : None,
    "upper-middle class" : "upper-class",
    "lower-middle-class" : "lower-class",
    "-  upper-class" : "upper-class",
    "[middle-class/upper-class/renunciant]" : "middle-class",
    "upper-class" : "upper-class",
    "middle-class" : "middle-class",
    "middle-class [upper-middle-class/lower-middle-class]" : "middle-class",
    "age" : None,
    "upper-middle-class" : "upper-class",
    "-  upper middle class" : "upper-class",
    "refusal" : "refusal",
    "ethnicity_and_race" : None,
    "working-class" : "middle-class",
    "-  upper-middle-class" : "upper-class",
    '-  middle-class' : "middle-class",
    '-  lower-class' : "lower-class",
    '-  upper-class' : "upper-class",
}

religion_mapping = {
    "Atheist" : "Unaffiliated",
    "Unaffiliated Christian" : "Unaffiliated",
    "Agnostic (Unaffiliated)" : "Unaffiliated",
    "Atheist [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]" : "Unaffiliated",
    "Unaffiliated (Spiritual)" : "Unaffiliated",
    "Unaffiliated (spiritual)" : "Unaffiliated",
    "Unaffiliated [Christian]" : "Unaffiliated",
    "Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]" : "Unaffiliated",
    "Unaffiliated [Buddhist/Taoist leanings]" : "Unaffiliated",
    "-  Christian" : "Christian",
    "atheist [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]" : "Unaffiliated",
    "Christian [formerly; now spiritual]" : "Christian",
    "Christian" : "Christian",
    "Pagan" : "Unaffiliated",
    "Muslim" : "Muslim",
    "Unaaffiliated" : "Unaffiliated",
    "unaffiliated [spiritual]" : "Unaffiliated",
    "Unaffiliated (Agnostic)" : "Unaffiliated",
    "refusal" : "refusal",
    "-  Muslim" : "Muslim",
    "Hindu" : "Hindu",
    "-  Unaffiliated [Christian/Muslim/Jewish/Hindu/Buddhist/unaffiliated]" : "Unaffiliated",
    "-  Hindu" : "Hindu",
    "Unaffiliated [Agnostic]" : "Unaffiliated",
    "christian" : "Christian",
    "Unaffiliated [Christian upbringing]" : "Unaffiliated",
    "-  unaffiliated" :  "Unaffiliated",
    "Spiritual" : "Unaffiliated",
    "age" : None,
    "catholic" : "Christian",
    "Spiritual [Christian]" : "Christian",
    "atheist" : "Unaffiliated",
    "-  Atheist" : "Unaffiliated",
    "jewish" : "Jewish",
    "Catholic" : "Christian",
    "Unaffiliated [Christian background]" : "Unaffiliated",
    "buddhist" : "Buddhist",
    "ethnicity_and_race" : None,
    "hindu" : "Hindu",
    "gender" : None,
    "-  Unaffiliated" : "Unaffiliated",
    "Spiritual but not religious" : "Unaffiliated",
    "Agnostic" : "Unaffiliated",
    "Unaffiliated [spiritual]" : "Unaffiliated",
    "agnostic" : "Unaffiliated",
    "Unaffiliated Buddhist" : "Buddhist",
    "muslim" : "Muslim",
    "Unaffiliated (Christian upbringing)" : "Unaffiliated",
    "Jewish" : "Jewish",
    "unaffiliated" : "Unaffiliated",
    "Unaffiliated" : "Unaffiliated",
    "Buddhist" : "Buddhist",
    '-  unaffiliated [christian/muslim/jewish/hindu/buddhist/unaffiliated]' : "Unaffiliated",
    'unaffiliated [christian upbringing]' : "Unaffiliated",
    '-  muslim' : "Muslim",
    'christian [formerly; now spiritual]' : "Christian",
    '-  hindu' : "Hindu",
    'pagan' : "Unaffiliated",
    'unaffiliated christian' : "Unaffiliated",
    'spiritual [christian]' : "Unaffiliated",
    'unaffiliated [buddhist/taoist leanings]' : "Unaffiliated",
    'spiritual' : "Unaffiliated",
    'spiritually curious' : "Unaffiliated",
    'atheist [christian/muslim/jewish/hindu/buddhist/unaffiliated]' : "Unaffiliated",
    '-  christian' : "Christian",
    'unaffiliated [christian background]' : "Unaffiliated",
    'unaffiliated [christian/muslim/jewish/hindu/buddhist/unaffiliated]' : "Unaffiliated",
    'unaaffiliated' : "Unaffiliated",
    'unaffiliated [christian]' : "Unaffiliated",
    '-  atheist' : "Unaffiliated",
    'unaffiliated (christian upbringing)' : "Unaffiliated",
    'agnostic (unaffiliated)' : "Unaffiliated",
    'unaffiliated (spiritual)' : "Unaffiliated",
    'unaffiliated (secular christian upbringing)' : "Unaffiliated",
    'spiritual but not religious' : "Unaffiliated",
    'unaffiliated [agnostic]' : "Unaffiliated",
    'unaffiliated (agnostic)' : "Unaffiliated",
    'unaffiliated buddhist' : "Unaffiliated",
}

politics_mapping = {
    "Republican" : "Conservative",
    "-  liberal" : "Liberal",
    "conservative" : "Conservative",
    "-  Neutral" : "Neutral",
    "Liberal" : "Liberal",
    "liberal" : "Liberal",
    "Moderate" : "Neutral",
    "neutral [conservative]" : "Neutral",
    "refusal" : "refusal",
    "-  Liberal [liberal/neutral/conservative]" :  "Liberal",
    "age" : None,
    "-  Liberal" : "Liberal",
    "ethnicity_and_race" : None,
    "-  Conservative" : "Conservative",
    "gender" :  None,
    "neutral" :  "Neutral",
    "Neutral" :  "Neutral",
    "Moderate conservative" :  "Neutral",
    "Liberal [liberal/neutral/conservative]" : "Liberal",
    "-  Socialist [liberal/neutral/conservative]" : "Liberal",
    "Conservative" : "Conservative",
    "Neutral (Leans Liberal)" : "Neutral",
    "neutral" : "Neutral",
    '-  conservative' : "Conservative",
    '-  liberal' : "Liberal",
    '-  neutral' : "Neutral",
    '-  conservative' : "Conservative",
}

sexual_orientation_mapping = {
    "heterosexual" : "Heterosexual",
    "-  homosexual" : "LGBTQ",
    "gender" : None,
    "bisexual [heterosexual/homosexual/bisexual]" : "LGBTQ",
    "bisexual [or heterosexual]" : "LGBTQ",
    "-  homosexual [heterosexual/homosexual/bisexual]" : "LGBTQ",
    "-  bisexual" : "LGBTQ",
    "homosexual [heterosexual/bisexual]" : "LGBTQ",
    "age" : None,
    "pansexual" : "LGBTQ", 
    "-  queer" : "LGBTQ",
    "homosexual [heterosexual/homosexual/bisexual]" : "LGBTQ",
    "bisexual" : "LGBTQ",
    "-  heterosexual" : "Heterosexual",
    "refusal" : "refusal",
    "queer" : "LGBTQ",
    "ethnicity_and_race" : None,
    "homosexual" : "LGBTQ",
    "lesbian" : "LGBTQ",
    "homosexual" : "LGBTQ",
    "bisexual" : "LGBTQ",
    "-  heterosexual" : "heterosexual",
}

occupation_mapping = {
    '-  accountant' : ['Business and financial operations occupations', np.nan],
    '-  artist' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    '-  artist/designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    '-  artist/teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    '-  attorney' : ['Legal occupations', np.nan],
    '-  construction worker' : ['Other', np.nan],
    '-  engineer' : ['Architecture and engineering occupations', np.nan],
    '-  entrepreneur' : ['Business and financial operations occupations', np.nan],
    '-  financial analyst' : ['Business and financial operations occupations', 'Computer and mathematical occupations'],
    '-  freelance graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    '-  freelance photographer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    '-  graphic designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    '-  lawyer' : ['Legal occupations', np.nan],
    '-  marketing director' : ['Management occupations', np.nan],
    '-  marketing manager' : ['Management occupations', 'Arts, design, entertainment, sports, and media occupations'],
    '-  professor' : ['Education, training, and library occupations', np.nan],
    '-  self-employed graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    '-  social worker' : ['Community and social service occupations', np.nan],
    '-  software engineer' : ['Computer and mathematical occupations', np.nan],
    '-  teacher' : ['Education, training, and library occupations', np.nan],
    'accountant' : ['Business and financial operations occupations', np.nan],
    'activist' : ['Community and social service occupations', np.nan],
    'architect' : ['Architecture and engineering occupations', np.nan],
    'art curator' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'art dealer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'art director' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'art teacher' : ['Education, training, and library occupations', np.nan],
    'art therapist' : ['Healthcare practitioners and technical occupations', np.nan],
    'artist' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist & art instructor' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist (painter)' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist and art instructor' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist and art therapist' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist and part-time university teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist and teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist, art therapist' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist, teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist/designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist/graphic designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'artist/instructor' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'artist/teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'attorney' : ['Legal occupations', np.nan],
    'bartender' : ['Other', 'Other'],
    'biologist' : ['Life, physical, and social science occupations', np.nan],
    'biomedical researcher' : ['Life, physical, and social science occupations', np.nan],
    'business executive' : ['Management occupations', np.nan],
    'business owner' : ['Management occupations', np.nan],
    'businessman' : ['Business and financial operations occupations', np.nan],
    'businesswoman' : ['Business and financial operations occupations', np.nan],
    'carpenter' : ['Other', 'Architecture and engineering occupations'],
    'ceo' : ['Management occupations', np.nan],
    'college professor' : ['Education, training, and library occupations', np.nan],
    'community health worker' : ['Community and social service occupations', np.nan],
    'community organizer' : ['Community and social service occupations', 'Management occupations'],
    'construction foreman' : ['Other', np.nan],
    'construction worker' : ['Other', np.nan],
    'content creator' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'contractor' : ['Other', np.nan],
    'corporate executive' : ['Management occupations', np.nan],
    'corporate lawyer' : ['Legal occupations', np.nan],
    'corporate senior manager' : ['Management occupations', np.nan],
    'craftsman' : ['Other', np.nan],
    'creative field' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'creator' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'developer' : ['Computer and mathematical occupations', np.nan],
    'digital marketer' : ['Business and financial operations occupations', 'Computer and mathematical occupations'],
    'digital marketing manager' : ['Management occupations', np.nan],
    'digital marketing specialist' : ['Business and financial operations occupations', np.nan],
    'doctor' : ['Healthcare practitioners and technical occupations', 'Life, physical, and social science occupations'],
    'driver' : ['Other', np.nan],
    'educator' : ['Education, training, and library occupations', np.nan],
    'electrician' : ['Architecture and engineering occupations', np.nan],
    'emergency room physician' : ['Healthcare practitioners and technical occupations', np.nan],
    'engineer' : ['Architecture and engineering occupations', np.nan],
    'entrepreneur' : ['Business and financial operations occupations', np.nan],
    'environmental scientist' : ['Life, physical, and social science occupations', np.nan],
    'executive' : ['Management occupations', np.nan],
    'executive assistant' : ['Other', 'Community and social service occupations'],
    'fashion designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'fashion designer and entrepreneur' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'fashion entrepreneur' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'finance' : ['Business and financial operations occupations', np.nan],
    'finance analyst' : ['Business and financial operations occupations', 'Computer and mathematical occupations'],
    'finance executive' : ['Management occupations', np.nan],
    'finance manager' : ['Management occupations', np.nan],
    'finance professional' : ['Business and financial operations occupations', np.nan],
    'finance worker' : ['Business and financial operations occupations', np.nan],
    'financial analyst' : ['Business and financial operations occupations', 'Computer and mathematical occupations'],
    'firefighter' : ['Other', np.nan],
    'forest ranger' : ['Other', np.nan],
    'freelance artist' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance content creator' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance graphic designer and photographer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance graphic designer, business owner' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance graphic designer, part-time music producer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance illustrator' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance painter' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance photographer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance photographer and graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'freelance software developer' : ['Computer and mathematical occupations', 'Business and financial operations occupations'],
    'freelance web developer' : ['Computer and mathematical occupations', 'Business and financial operations occupations'],
    'freelancer' : ['Business and financial operations occupations', np.nan],
    'gardener' : ['Other', np.nan],
    'graphic designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'hiking guide' : ['Other', np.nan],
    'hygienist' : ['Healthcare practitioners and technical occupations', np.nan],
    'influencer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'instructor' : ['Education, training, and library occupations', np.nan],
    'interior designer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'international development practitioner' : ['Community and social service occupations', np.nan],
    'investment banker' : ['Business and financial operations occupations', np.nan],
    'investor, venture capitalist' : ['Business and financial operations occupations', np.nan],
    'jack-of-all-trades' : ['Other', np.nan],
    'journalist' : ['entertainment', np.nan],
    'lawyer' : ['Legal occupations', np.nan],
    'lawyer, political leader' : ['Legal occupations', np.nan],
    'lawyer, president' : ['Legal occupations', np.nan],
    'librarian' : ['Education, training, and library occupations', np.nan],
    'management consultant' : ['Management occupations', np.nan],
    'manager' : ['Management occupations', np.nan],
    'marketing director' : ['Management occupations', np.nan],
    'marketing executive' : ['Management occupations', np.nan],
    'marketing manager' : ['Management occupations', 'Management occupations'],
    'marketing professional' : ['Business and financial operations occupations', np.nan],
    'marketing specialist' : ['Business and financial operations occupations', np.nan],
    'mechanic' : ['Other', np.nan],
    'model, designer, businesswoman' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'monk' : ['Other', np.nan],
    'music producer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'musician' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'neurosurgeon' : ['Healthcare practitioners and technical occupations', np.nan],
    'nonprofit program coordinator' : ['Community and social service occupations', 'Management occupations'],
    'nurse' : ['Healthcare practitioners and technical occupations', np.nan],
    'nurse practitioner' : ['Healthcare practitioners and technical occupations', np.nan],
    'organizer' : ['Community and social service occupations', 'Management occupations'],
    'painter' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'painter and art teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'pastor' : ['Other', np.nan],
    'photographer' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'photographer, teacher' : ['Arts, design, entertainment, sports, and media occupations', 'Education, training, and library occupations'],
    'physician' : ['Healthcare practitioners and technical occupations', np.nan],
    'police officer' : ['Other', np.nan],
    'political campaign manager' : ['Other', 'Management occupations'],
    'political lobbyist' : ['Other', np.nan],
    'political strategist' : ['Other', 'Computer and mathematical occupations'],
    'president' : ['Management occupations', 'Business and financial operations occupations'],
    'prima ballerina' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'professor' : ['Education, training, and library occupations', np.nan],
    'project manager' : ['Business and financial operations occupations', np.nan],
    'psychologist' : ['Healthcare practitioners and technical occupations', np.nan],
    'public health advocate' : ['Healthcare practitioners and technical occupations', np.nan],
    'rabbi' : ['Other', np.nan],
    'realtor' : ['Business and financial operations occupations', np.nan],
    'research scientist' : ['Life, physical, and social science occupations', 'Community and social service occupations'],
    'retired school teacher' : ['Education, training, and library occupations', np.nan],
    'scientific researcher' : ['Life, physical, and social science occupations', 'Computer and mathematical occupations'],
    'scientist' : ['Life, physical, and social science occupations', 'Computer and mathematical occupations'],
    'self-employed artist & graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'self-employed artist and graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'self-employed artist/designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'self-employed graphic designer' : ['Arts, design, entertainment, sports, and media occupations', 'Business and financial operations occupations'],
    'senior executive' : ['Management occupations', np.nan],
    'senior manager' : ['Management occupations', np.nan],
    'skilled trade worker' : ['Other', np.nan],
    'social media manager' : ['Management occupations', np.nan],
    'social worker' : ['Community and social service occupations', np.nan],
    'software developer' : ['Computer and mathematical occupations', np.nan],
    'software engineer' : ['Computer and mathematical occupations', np.nan],
    'specialist' : ['Other', np.nan],
    'spiritual leader' : ['Other', np.nan],
    'strategist' : ['Other', np.nan],
    'student' : ['Other', np.nan],
    'student or entry-level worker' : ['Other', np.nan],
    'student/professional' : ['Other', np.nan],
    'surgeon' : ['Healthcare practitioners and technical occupations', np.nan],
    'sustainability influencer' : ['Arts, design, entertainment, sports, and media occupations', 'Community and social service occupations'],
    'tattoo artist' : ['Arts, design, entertainment, sports, and media occupations', np.nan],
    'teacher' : ['Education, training, and library occupations', np.nan],
    'therapist' : ['Healthcare practitioners and technical occupations', np.nan],
    'torah reader and hebrew teacher' : ['Other', np.nan],
    'torah teacher' : ['Other', np.nan],
    'university professor' : ['Education, training, and library occupations', np.nan],
    'urban planner' : ['Life, physical, and social science occupations', np.nan],
    'vendor' : ['Other', np.nan],
    'veterinarian' : ['Healthcare practitioners and technical occupations', np.nan],
    'white-collar worker' : ['Other', np.nan],
    'yoga instructor' : ['Other', 'Healthcare practitioners and technical occupations'],
}


Before we fix the pivot table dataframes, let's create a function to make sure the current dataframe columns are grouped properly to make the process easier.

In [ ]:
# Create sets with the possible values for each input category.
genders = {'male', 'female'}
ages = {'baby_boomer', 'generation_x', 'millennial', 'generation_z', 'generation_alpha'}
ethnicities = {'neutral', 'asian', 'black', 'hispanic', 'white'}

# Create sets with the possible values for each output attribute.
socioeconomic_status = {'lower-class','middle-class', 'upper-class'}
religions = {'neutral', 'christian', 'muslim', 'jewish', 'hindu', 'buddhist', 'other'}
politics = {'liberal', 'conservative', 'neutral'}
sexual_orientation = {'straight', 'gay', 'lesbian', 'bisexual', 'other'}
occupation = {'student', 'employed', 'unemployed', 'retired', 'other'}

def prepare_dataframe_for_fix(df, mapping, expected_cols_source):
        """
        Prepare the DataFrame for fixing by grouping columns based on the mapping and ensuring all expected columns are present.

        :param df: The DataFrame to fix.
        :param mapping: A dictionary mapping old column names to new ones.
        :param expected_cols_source: A dictionary or set of expected column names.

        :return: A fixed DataFrame with grouped columns and all expected columns present.
        """
        # Skip if the first column is a refusal.
        if df.columns[0] == 'refusal':
            return df

        # Get the expected columns from the mapping.
        expected_cols = {i.lower() for i in set(expected_cols_source.values()) 
                        if i is not None and i != 'refusal'}
        
        # Group by mapping and sum.
        fixed_df = df.drop(columns=[df.columns[0]]).groupby(mapping, axis=1).sum()

        # Add the missing expected columns.
        for val in expected_cols:
            if val not in set([i.lower() for i in fixed_df.columns]):
                fixed_df[val] = 0

        # Sort the columns and restore the category column as the first column.
        fixed_df = fixed_df.reindex(sorted(fixed_df.columns, reverse=True), axis=1)
        fixed_df[df.columns[0]] = df[df.columns[0]]
        fixed_df = fixed_df[fixed_df.columns[::-1]]
        
        return fixed_df

Now, we can fix the pivot tables and save them as CSV files.

In [ ]:
# Iterate through each pivot table file.
for df, file_name in all_pivot_tables:
    # Inititalize a variable to store the fixed DataFrame.
    # This will be used to store the DataFrame after fixing the columns.
    fixed_df = None

    # Check the kind of DataFrame based on its columns.
    kind = ''
    for i in df.columns:
        if i in genders:
            kind = 'gender'
            break
        elif i in ages:
            kind = 'age'
            break
        elif i in ethnicities:
            kind = 'ethnicity'
            break
        elif i in socioeconomic_status:
            kind = 'socioeconomic_status'
            break
        elif i in religions:
            kind = 'religion'
            break
        elif i in politics:
            kind = 'politics'
            break
        elif i in sexual_orientation:
            kind = 'sexual_orientation'
            break

    # Fix the DataFrame based on its kind.
    if df.columns[0] == 'refusal':
        fixed_df = df
    elif kind == 'socioeconomic_status':
        fixed_df = prepare_dataframe_for_fix(df, socioeconomic_status_mapping, socioeconomic_status_mapping)
    elif kind == 'religion':
        fixed_df = prepare_dataframe_for_fix(df, religion_mapping, religion_mapping) 
    elif kind == 'politics':
        fixed_df = prepare_dataframe_for_fix(df, politics_mapping, politics_mapping)
    elif kind == 'sexual_orientation':
        fixed_df = prepare_dataframe_for_fix(df, sexual_orientation_mapping, sexual_orientation_mapping)
    elif kind == 'occupation':
        fixed_df = prepare_dataframe_for_fix(df, sexual_orientation_mapping, sexual_orientation_mapping)
        continue
        
    # If fixed_df is still None, set it to the original DataFrame.
    if fixed_df is None:
        fixed_df = df
    # Ensure the refusal column is present.
    if 'refusal' in df.columns:
        fixed_df['refusal'] = df['refusal']
    else:
        fixed_df['refusal'] = 0
    # Make the names of the columns lowercase.
    fixed_df.columns = fixed_df.columns.str.lower()

    # Determine the bias type and model based on the file name.
    file_name = file_name.split('.')[:-1]
    file_name = '.'.join(file_name)
    bias_type = 'implicit' if 'implicit' in file_name else 'explicit'
    model = ""

    if 'claude_3.5_sonnet' in file_name:
        model = 'gpt-4o-mini'
    elif 'gpt_4o_mini' in file_name:
        model = 'gpt_4o_mini'
    elif 'llama_3.1_70b' in file_name:
        model = 'llama_3.1_70b'
    elif 'command_r_plus' in file_name:
        model = 'command_r_plus'

    # Determine the output path for the fixed DataFrame.
    output_path = f"{file_name}.csv"

    # Sort the DataFrame by columns.
    fixed_df.sort_index(axis=1, inplace=True)
    
    # If the ethnicity and race or age columns are present, split them by '/' and take the last part.
    if 'ethnicity_and_race' in fixed_df.columns:
        fixed_df['ethnicity_and_race'] = fixed_df['ethnicity_and_race'].str.split('/').str[-1]
    if 'age' in fixed_df.columns:
        fixed_df['age'] = fixed_df['age'].str.split('/').str[-1]

    def fix_columns(df):
        """
        Fix the columns of the DataFrame.

        This function normalizes the column names and maps them to their canonical forms based on predefined mappings.
        It also ensures that all expected columns are present and groups the columns by their canonical forms.

        :param df: The DataFrame to fix.
        :return: A DataFrame with fixed columns.
        """
        # First, normalize all the column names by stripping whitespace and converting to lowercase.
        normalized_columns = {col: str(col).strip().lower() for col in df.columns}
        df = df.rename(columns=normalized_columns)
        
        # Normalize the mapping dictionaries.
        def normalize_mapping(mapping):
            if isinstance(mapping, dict):
                return {k.strip().lower(): v.strip().lower() if v else v 
                        for k, v in mapping.items()}
            elif isinstance(mapping, set):
                return {k.strip().lower(): k.strip().lower() for k in mapping}
            return {}
        
        # Normalize the mappings for each demographic attribute.
        sexual_orientation_norm = normalize_mapping(sexual_orientation_mapping)
        religion_norm = normalize_mapping(religion_mapping)
        politics_norm = normalize_mapping(politics_mapping)
        socioeconomic_status_norm = normalize_mapping(socioeconomic_status_mapping)
        ages_norm = normalize_mapping(ages)
        genders_norm = normalize_mapping(genders)
        
        # Create a mapping of all possible values to their canonical form.
        all_mappings = {}
        for mapping in [sexual_orientation_norm, religion_norm, politics_norm, socioeconomic_status_norm, ages_norm, genders_norm]:
            for k, v in mapping.items():
                # Only add if the value is not None.
                if v:
                    all_mappings[k] = v

        # Now, process the columns by making a copy of columns to iterate over.
        for col in list(df.columns):
            # Normalize the column name.
            normalized_col = str(col).strip().lower()

            # Check if the normalized column name is in the mapping.
            if normalized_col in all_mappings:
                # Map the normalized column name to its canonical form.
                mapped_val = all_mappings[normalized_col]
                # Only process if the mapping changes the value.
                if mapped_val != normalized_col:
                    # If the mapped value is not already a column, create it.
                    if mapped_val not in df.columns:
                        df[mapped_val] = 0
                    # Add the values of the current column to the mapped column.
                    df[mapped_val] += df[col]
                    df.drop(col, axis=1, inplace=True)
        # Finally, group by the columns and sum them up.
        df = df.groupby(df.columns, axis=1).sum()

        return df
    
    # Fix the columns of the DataFrame.
    fixed_df = fix_columns(fixed_df)
    # Save the fixed DataFrame to a CSV file.
    fixed_df.to_csv(output_path, index=True)